In [1]:
import numpy as np
import polars as pl
from pathlib import Path

import matplotlib.pyplot as plt
plt.rcParams["font.size"] = 16

# 2.A : Few trajectories + Mean + Theory (Homogeneous)

In [15]:
# --- Load files --- #
root = Path("//home/nicolas/Documents/Workspace/nucleo/outputs/2026-02-24__PC/nucleo__fig_0")
paths = [str(p) for p in root.glob("*/**/*.parquet")] or [str(p) for p in root.glob("*/*.parquet")]
df_all = (
    pl.scan_parquet(paths)
    .collect()
    .sort(by=[
        "algorithm", "fact", "factmode",
        "landscape", "s", "l", "lmbda", "alphar"
        ])
)
thetas = np.unique(df_all["theta"].to_numpy())
# print(df_all)


# --- Figure --- #
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(16,6), dpi=1200)

for idx, theta_val in enumerate(thetas[0:2]):
    df_theta = df_all.filter(pl.col("theta") == theta_val)
    trajs = df_theta["results"][0].to_numpy()
    mean_traj = df_theta["results_mean"][0].to_numpy()
    v_mean_th = df_theta["v_mean_th"].to_numpy()
    times = df_theta["times"][0].to_numpy()
    mu = df_theta["mu"].to_numpy()

    ax[idx].plot(trajs[0], drawstyle="steps-post", color="red", alpha=0.30, label="trajectories")
    for i in range(1, 10):
        ax[idx].plot(trajs[i], drawstyle="steps-post", color="red", alpha=0.30)
    ax[idx].plot(mean_traj, color="red", ls="-", lw=2, label=r"$v_{mean}$")
    ax[idx].scatter(times, times*v_mean_th, color="black", lw=2, label=r"$v_{homogeneous}$", marker="+")
    ax[idx].set_ylim([-100, 2100])
    ax[idx].grid(True, alpha=0.3)
    ax[idx].set_title(f"(μ,θ) = ({mu[0]},{theta_val})")
    ax[idx].legend()

# étiquettes
ax[0].set_ylabel("position")
ax[0].set_xlabel("time")
ax[1].set_xlabel("time")
# ax[2].set_xlabel("time")

plt.tight_layout()
plt.show()

# 2.B : Their distributions of speeds

In [28]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

# --- Figure --- #
plt.figure(figsize=(8,6), dpi=1200)
plt.title("Homogeneous")

# Palette de verts, seconde moitié du colormap "Greens"
cmap = cm.get_cmap("Greens")
n_colors = len(thetas[0:2])
# Prendre les couleurs entre 0.5 et 0.9 du cmap (second half)
color_list = [cmap(0.5 + 0.4*i/(n_colors-1)) for i in range(n_colors)]

for idx, theta_val in enumerate(thetas[0:2]):

    df_theta = df_all.filter(pl.col("theta") == theta_val)
    vi_points = df_theta["vi_points"][0].to_numpy()
    vi_distrib = df_theta["vi_distrib"][0].to_numpy()
    v_mean = df_theta["v_mean"].to_numpy()
    print(v_mean)

    plt.plot(
        vi_points,
        vi_distrib,
        color=color_list[idx],
        ls="-",
        lw=2,
        label=f"(μ,θ) = ({mu[0]},{theta_val})"
    )

    plt.axvline(
        x=v_mean,
        color=color_list[idx],
        ls="--",
        lw=2,
        label = rf"$v_{{mean}}$ = {v_mean[0]:.2f}"
    )

plt.xlabel(r'speeds in ($\sigma k_0$) unit')
plt.ylabel("Distribution")
plt.xlim([1e-1, 1e3])

plt.xscale("log")
plt.yscale("log")
plt.grid(True, alpha=0.3, which="both")
plt.legend()
plt.show()

/tmp/ipykernel_46225/3897794634.py:10: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap("Greens")


[12.51246805]
[12.52387097]
